In [1]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)

In [6]:
from quanta.clients.yfinance import YahooFinanceClient
from datetime import datetime, timedelta

n=365
from_date = datetime.now() - timedelta(days=n)
to_date = datetime.now() - timedelta(days=1)


yh = YahooFinanceClient()
df_init = yh.get_price(
    "^SPMCBRT",
    from_date=from_date.strftime("%Y-%m-%d"),
    to_date=to_date.strftime("%Y-%m-%d"),
    interval="1h", 
    postclean=True
)
df_init.to_pandas()

,timestamp,datetime,open,high,low,close,volume
0,1730903400,2024-11-06 15:30:00,1261.956909,1284.460327,1261.956909,1280.506226,0
1,1730907000,2024-11-06 16:30:00,1279.447144,1293.914307,1275.322266,1287.995850,0
2,1730910600,2024-11-06 17:30:00,1286.996704,1288.534058,1277.593994,1284.306641,0
3,1730914200,2024-11-06 18:30:00,1284.648193,1285.451050,1280.036499,1281.701782,0
4,1730917800,2024-11-06 19:30:00,1281.505371,1282.940063,1274.126709,1276.389771,0
...,...,...,...,...,...,...,...
1486,1762270200,2025-11-04 16:30:00,1245.520264,1250.555054,1245.044922,1248.071655,0
1487,1762273800,2025-11-04 17:30:00,1247.441040,1250.157349,1246.306152,1248.915649,0
1488,1762277400,2025-11-04 18:30:00,1248.226807,1251.515381,1246.257446,1249.963257,0
1489,1762281000,2025-11-04 19:30:00,1249.992432,1250.855835,1245.073975,1246.703857,0


In [8]:
from quanta.utils.ta import SMA, RSI, MACD, TAClient
indicators = [
    SMA(10),
    SMA(20),
    RSI(14),
    MACD() 
]
ta_client = TAClient()
df = ta_client.calculate_indicators(df_init, indicators)

In [9]:
from quanta.utils.trace import Candlesticks, Volume, Line

traces = [
    Candlesticks(),
    Line('hl_avg', name='HL Average', color='purple'),
    Volume()
]

In [10]:
from quanta.clients.chart import ChartClient

chart_client = ChartClient()
chart_client.plot(df, "AAPL", traces=traces, indicators=indicators, theme='professional')

Plotting 1491 bars for AAPL with x_axis_type='row_nb'


In [11]:
# test baysian optimization with skopt

import polars as pl
import numpy as np

def backtest_strategy(df: pl.DataFrame, short_window: int, long_window: int, rsi_period: int, stop_loss: float, take_profit: float) -> dict:
    """
    Version avec gestion d'erreurs robuste
    """
    try:
        df = df.clone().drop_nulls()
        
        if len(df) == 0:
            return {
                "total_return": 0.0,
                "sharpe_ratio": 0.0,
                "cagr": 0.0,
                "max_drawdown": 0.0
            }
        
        # Vérifier que les colonnes nécessaires existent
        required_cols = [f'SMA_{short_window}', f'SMA_{long_window}', f'RSI_{rsi_period}', 'close']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if missing_cols:
            raise ValueError(f"Colonnes manquantes: {missing_cols}")
        
        # Générer les signaux
        df = df.with_columns([
            pl.when(
                (pl.col(f'SMA_{short_window}') > pl.col(f'SMA_{long_window}')) & 
                (pl.col(f'RSI_{rsi_period}') < 30)
            ).then(1)
            .when(
                (pl.col(f'SMA_{short_window}') < pl.col(f'SMA_{long_window}')) & 
                (pl.col(f'RSI_{rsi_period}') > 70)
            ).then(-1)
            .otherwise(0)
            .alias('signal')
        ])
        
        # Calculer les rendements
        df = df.with_columns([
            pl.col('close').pct_change().alias('price_returns'),
            pl.col('signal').shift(1).fill_null(0).alias('signal_shifted')
        ])
        
        df = df.with_columns([
            (pl.col('price_returns').fill_null(0) * pl.col('signal_shifted') - (stop_loss + take_profit))
            .alias('strategy_returns')
        ])
        
        # Calculer les métriques
        returns = df['strategy_returns'].drop_nulls()
        
        if len(returns) == 0:
            return {
                "total_return": 0.0,
                "sharpe_ratio": 0.0,
                "cagr": 0.0,
                "max_drawdown": 0.0
            }
        
        total_return = returns.sum()
        mean_ret = returns.mean()
        std_ret = returns.std()
        
        # Sharpe ratio
        sharpe_ratio = (mean_ret / std_ret) * np.sqrt(252 * 390) if std_ret and std_ret > 0 else 0.0
        
        # CAGR
        close_first = df['close'][0]
        close_last = df['close'][-1]
        n_periods = len(df)
        cagr = (close_last / close_first) ** (252 * 390 / n_periods) - 1 if n_periods > 0 and close_first > 0 else 0.0
        
        # Max Drawdown
        df = df.with_columns([
            ((1 + pl.col('strategy_returns')).cum_prod()).alias('cum_returns')
        ])
        df = df.with_columns([
            (pl.col('cum_returns') / pl.col('cum_returns').cum_max() - 1).alias('drawdown')
        ])
        
        max_drawdown = df['drawdown'].min() or 0.0
        
        return {
            "total_return": float(total_return),
            "sharpe_ratio": float(sharpe_ratio),
            "cagr": float(cagr),
            "max_drawdown": float(max_drawdown)
        }
        
    except Exception as e:
        print(f"Erreur dans backtest_strategy: {e}")
        return {
            "total_return": 0.0,
            "sharpe_ratio": 0.0,
            "cagr": 0.0,
            "max_drawdown": 0.0
        }

def objective(params):
    short_window, long_window, rsi_period, stop_loss, take_profit = params
    indicators = [
        SMA(short_window),
        SMA(long_window),
        RSI(),
    ]
    ta_client = TAClient()
    df = ta_client.calculate_indicators(df_init, indicators)

    # Backtest the strategy
    metrics = backtest_strategy(df, short_window, long_window, rsi_period, stop_loss, take_profit)
    
    # return minimize negative sharpe ratio
    return -metrics['sharpe_ratio']
    
  


In [12]:
# Configuration JSON avec structure logique
OPTIMIZATION_CONFIG = {
    "indicators": {
        "SMA_short": {
            "class": "SMA",
            "params": {
                "period": {
                    "type": "int",
                    "low": 5,
                    "high": 50
                }
            }
        },
        "SMA_long": {
            "class": "SMA",
            "params": {
                "period": {
                    "type": "int",
                    "low": 20,
                    "high": 200
                }
            }
        },
        "RSI": {
            "class": "RSI",
            "params": {
                "period": {
                    "type": "int",
                    "low": 7,
                    "high": 21
                }
            }
        },
        "BollingerBands": {
            "class": "BollingerBands",
            "params": {
                "period": {
                    "type": "int",
                    "low": 10,
                    "high": 30
                },
                "std_dev": {
                    "type": "float",
                    "low": 1.5,
                    "high": 3.0
                }
            }
        },
        "MACD": {
            "class": "MACD",
            "params": {
                "fast_period": {
                    "type": "int",
                    "low": 8,
                    "high": 16
                },
                "slow_period": {
                    "type": "int",
                    "low": 20,
                    "high": 30
                },
                "signal_period": {
                    "type": "int",
                    "low": 7,
                    "high": 12
                }
            }
        }
    },
    "strategy": {
        "stop_loss": {
            "type": "float",
            "low": 0.001,
            "high": 0.05,
            "log": True
        },
        "take_profit": {
            "type": "float",
            "low": 0.001,
            "high": 0.05,
            "log": True
        },
        "position_size": {
            "type": "float",
            "low": 0.1,
            "high": 1.0
        }
    },
    "constraints": [
        {
            "condition": "SMA_short_period < SMA_long_period"
        },
        {
            "condition": "MACD_fast_period < MACD_slow_period"
        }
    ],
    "optimization": {
        "direction": "maximize",
        "n_trials": 100,
        "n_jobs": 1,
        "timeout": 3600,
        "seed": 42
    }
}

In [13]:
from quanta.utils.ta import SMA, EMA, RSI, MACD, BollingerBands, ATR, Stochastic

# Mapping des classes d'indicateurs
INDICATOR_CLASSES = {
    'SMA': SMA,
    'EMA': EMA,
    'RSI': RSI,
    'MACD': MACD,
    'BollingerBands': BollingerBands,
    'ATR': ATR,
    'Stochastic': Stochastic,
}

In [14]:
from typing import Any, Dict, List
import optuna

def suggest_params_from_config(trial: optuna.Trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Génère les suggestions de paramètres pour les indicateurs et la stratégie
    
    Args:
        trial: Objet trial d'Optuna
        config: Configuration JSON complète
    
    Returns:
        Dictionnaire avec 'indicators' et 'strategy'
    """
    params = {
        'indicators': {},
        'strategy': {}
    }
    
    # Suggérer les paramètres des indicateurs
    for indicator_name, indicator_config in config.get("indicators", {}).items():
        params['indicators'][indicator_name] = {}
        
        for param_name, param_config in indicator_config.get("params", {}).items():
            param_type = param_config["type"]
            full_param_name = f"{indicator_name}_{param_name}"
            
            if param_type == "int":
                value = trial.suggest_int(
                    full_param_name,
                    param_config["low"],
                    param_config["high"],
                    step=param_config.get("step", 1)
                )
            elif param_type == "float":
                value = trial.suggest_float(
                    full_param_name,
                    param_config["low"],
                    param_config["high"],
                    log=param_config.get("log", False),
                    step=param_config.get("step", None)
                )
            elif param_type == "categorical":
                value = trial.suggest_categorical(
                    full_param_name,
                    param_config["choices"]
                )
            
            params['indicators'][indicator_name][param_name] = value
    
    # Suggérer les paramètres de stratégie
    for param_name, param_config in config.get("strategy", {}).items():
        param_type = param_config["type"]
        
        if param_type == "int":
            value = trial.suggest_int(
                param_name,
                param_config["low"],
                param_config["high"],
                step=param_config.get("step", 1)
            )
        elif param_type == "float":
            value = trial.suggest_float(
                param_name,
                param_config["low"],
                param_config["high"],
                log=param_config.get("log", False),
                step=param_config.get("step", None)
            )
        elif param_type == "categorical":
            value = trial.suggest_categorical(
                param_name,
                param_config["choices"]
            )
        
        params['strategy'][param_name] = value
    
    return params

def create_indicators_from_params(config: Dict[str, Any], params: Dict[str, Any]) -> List:
    """
    Crée automatiquement les indicateurs depuis les paramètres
    
    Args:
        config: Configuration JSON complète
        params: Paramètres suggérés (avec structure 'indicators' et 'strategy')
    
    Returns:
        Liste des objets indicateurs instanciés
    """
    indicators = []
    
    for indicator_name, indicator_params in params['indicators'].items():
        indicator_config = config['indicators'][indicator_name]
        indicator_class_name = indicator_config['class']
        
        if indicator_class_name not in INDICATOR_CLASSES:
            raise ValueError(f"Indicateur '{indicator_class_name}' non trouvé dans INDICATOR_CLASSES")
        
        indicator_class = INDICATOR_CLASSES[indicator_class_name]
        
        # Instancier l'indicateur avec les paramètres
        try:
            # Essayer d'abord avec kwargs
            indicator = indicator_class(**indicator_params)
        except TypeError:
            # Si ça échoue, essayer avec args positionnels
            indicator = indicator_class(*indicator_params.values())
        
        indicators.append(indicator)
    
    return indicators

def check_constraints(params: Dict[str, Any], constraints: list) -> bool:
    """
    Vérifie si les paramètres respectent les contraintes
    
    Args:
        params: Paramètres suggérés (structure plate pour l'évaluation)
        constraints: Liste des contraintes
    
    Returns:
        True si toutes les contraintes sont respectées
    """
    # Aplatir la structure des params pour l'évaluation
    flat_params = {}
    
    # Ajouter les paramètres d'indicateurs
    for indicator_name, indicator_params in params['indicators'].items():
        for param_name, param_value in indicator_params.items():
            flat_params[f"{indicator_name}_{param_name}"] = param_value
    
    # Ajouter les paramètres de stratégie
    flat_params.update(params['strategy'])
    
    # Vérifier les contraintes
    for constraint in constraints:
        condition = constraint["condition"]
        try:
            if not eval(condition, {}, flat_params):
                return False
        except Exception as e:
            print(f"Erreur lors de l'évaluation de la contrainte '{condition}': {e}")
            return False
    
    return True

/home/cmileto/miniconda3/envs/ta/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [15]:
from optuna.samplers import TPESampler
import json

def objective(trial):
    """
    Fonction objectif pour Optuna avec configuration JSON
    """
    config = OPTIMIZATION_CONFIG
    
    # Suggérer les paramètres depuis la config
    params = suggest_params_from_config(trial, config)
    
    # Vérifier les contraintes
    if not check_constraints(params, config.get("constraints", [])):
        return float('-inf')
    
    # Créer automatiquement les indicateurs depuis les paramètres
    indicators = create_indicators_from_params(config, params)
    
    print(indicators)
    
    # Debug: afficher les paramètres testés
    print(f"\nTrial {trial.number}:")
    print(f"  Indicators: {params['indicators']}")
    print(f"  Strategy: {params['strategy']}")
    
    # Calculer les indicateurs sur le DataFrame
    ta_client = TAClient()
    df = ta_client.calculate_indicators(df_init, indicators)
    
    # Backtest de la stratégie
    try:
        # Extraire les paramètres nécessaires pour le backtest
        # Adapter selon votre fonction backtest_strategy
        short_window = params['indicators']['SMA_short']['period']
        long_window = params['indicators']['SMA_long']['period']
        rsi_period = params['indicators']['RSI']['period']
        stop_loss = params['strategy']['stop_loss']
        take_profit = params['strategy']['take_profit']
        
        metrics = backtest_strategy(
            df, 
            short_window,
            long_window,
            rsi_period,
            stop_loss,
            take_profit
        )
        
        # Enregistrer les métriques
        trial.set_user_attr('total_return', metrics['total_return'])
        trial.set_user_attr('cagr', metrics['cagr'])
        trial.set_user_attr('max_drawdown', metrics['max_drawdown'])
        
        return metrics['sharpe_ratio']
    
    except Exception as e:
        print(f"  ❌ Erreur: {e}")
        return float('-inf')

def run_optimization(config: Dict[str, Any]):
    """
    Lance l'optimisation avec la configuration fournie
    """
    opt_config = config["optimization"]
    
    # Afficher la configuration
    print("="*70)
    print("CONFIGURATION DE L'OPTIMISATION")
    print("="*70)
    print(f"\nIndicateurs à optimiser:")
    for ind_name, ind_config in config['indicators'].items():
        print(f"  - {ind_name} ({ind_config['class']}): {list(ind_config['params'].keys())}")
    
    print(f"\nParamètres de stratégie:")
    for param_name in config['strategy'].keys():
        print(f"  - {param_name}")
    
    print(f"\nNombre de trials: {opt_config['n_trials']}")
    print(f"Direction: {opt_config['direction']}")
    print("="*70 + "\n")
    
    study = optuna.create_study(
        direction=opt_config["direction"],
        study_name='trading_strategy_optimization',
        sampler=TPESampler(seed=opt_config.get("seed", None)),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10)
    )
    
    study.optimize(
        objective, 
        n_trials=opt_config["n_trials"],
        n_jobs=opt_config.get("n_jobs", 1),
        show_progress_bar=True,
        timeout=opt_config.get("timeout", None)
    )
    
    print("\n" + "="*70)
    print("RÉSULTATS DE L'OPTIMISATION")
    print("="*70)
    print(f"\n🏆 Meilleur Sharpe Ratio: {study.best_value:.4f}")
    
    print(f"\n📊 Meilleurs paramètres:")
    print("\n  Indicateurs:")
    for key, value in study.best_params.items():
        if any(ind_name in key for ind_name in config['indicators'].keys()):
            print(f"    {key}: {value}")
    
    print("\n  Stratégie:")
    for key, value in study.best_params.items():
        if key in config['strategy'].keys():
            print(f"    {key}: {value}")
    
    print(f"\n💰 Métriques de performance:")
    print(f"    Total Return: {study.best_trial.user_attrs['total_return']:.4f}")
    print(f"    CAGR: {study.best_trial.user_attrs['cagr']:.4f}")
    print(f"    Max Drawdown: {study.best_trial.user_attrs['max_drawdown']:.4f}")
    
    return study


# Lancer l'optimisation
if __name__ == "__main__":
    # Option 1: Utiliser la config directement
    study = run_optimization(OPTIMIZATION_CONFIG)
    
    # Sauvegarder les résultats
    study.trials_dataframe().to_csv('optimization_results.csv', index=False)
    
    # Sauvegarder la meilleure config
    best_config = {
        'indicators': {},
        'strategy': {}
    }
    
    for key, value in study.best_params.items():
        if any(ind_name in key for ind_name in OPTIMIZATION_CONFIG['indicators'].keys()):
            # C'est un paramètre d'indicateur
            for ind_name in OPTIMIZATION_CONFIG['indicators'].keys():
                if key.startswith(ind_name):
                    if ind_name not in best_config['indicators']:
                        best_config['indicators'][ind_name] = {}
                    param_name = key.replace(f"{ind_name}_", "")
                    best_config['indicators'][ind_name][param_name] = value
        else:
            # C'est un paramètre de stratégie
            best_config['strategy'][key] = value
    
    with open('best_config.json', 'w') as f:
        json.dump(best_config, f, indent=4)
    
    print("\n✅ Meilleure configuration sauvegardée dans 'best_config.json'")

[I 2025-11-06 17:41:29,181] A new study created in memory with name: trading_strategy_optimization


CONFIGURATION DE L'OPTIMISATION

Indicateurs à optimiser:
  - SMA_short (SMA): ['period']
  - SMA_long (SMA): ['period']
  - RSI (RSI): ['period']
  - BollingerBands (BollingerBands): ['period', 'std_dev']
  - MACD (MACD): ['fast_period', 'slow_period', 'signal_period']

Paramètres de stratégie:
  - stop_loss
  - take_profit
  - position_size

Nombre de trials: 100
Direction: maximize



Best trial: 0. Best value: 0:  19%|█▉        | 19/100 [00:00<00:00, 115.23it/s, 0.19/3600 seconds]

[<quanta.utils.ta.SMA object at 0x715a961ab890>, <quanta.utils.ta.SMA object at 0x715a9630d910>, <quanta.utils.ta.RSI object at 0x715a961aae90>, <quanta.utils.ta.BollingerBands object at 0x715a961ab910>, <quanta.utils.ta.MACD object at 0x715a9630e350>]

Trial 0:
  Indicators: {'SMA_short': {'period': 22}, 'SMA_long': {'period': 192}, 'RSI': {'period': 17}, 'BollingerBands': {'period': 22, 'std_dev': 1.7340279606636548}, 'MACD': {'fast_period': 9, 'slow_period': 20, 'signal_period': 12}}
  Strategy: {'stop_loss': 0.010502105436744277, 'take_profit': 0.01595857358814127, 'position_size': 0.1185260448662222}
Erreur dans backtest_strategy: Colonnes manquantes: ['SMA_22', 'SMA_192', 'RSI_17']
[I 2025-11-06 17:41:29,199] Trial 0 finished with value: 0.0 and parameters: {'SMA_short_period': 22, 'SMA_long_period': 192, 'RSI_period': 17, 'BollingerBands_period': 22, 'BollingerBands_std_dev': 1.7340279606636548, 'MACD_fast_period': 9, 'MACD_slow_period': 20, 'MACD_signal_period': 12, 'stop_loss'

Best trial: 0. Best value: 0:  34%|███▍      | 34/100 [00:00<00:00, 81.74it/s, 0.40/3600 seconds] 

...

[<quanta.utils.ta.SMA object at 0x715a960bee10>, <quanta.utils.ta.SMA object at 0x715a960becd0>, <quanta.utils.ta.RSI object at 0x715a95ee5ed0>, <quanta.utils.ta.BollingerBands object at 0x715a95ee58d0>, <quanta.utils.ta.MACD object at 0x715a95ee6010>]

Trial 62:
  Indicators: {'SMA_short': {'period': 33}, 'SMA_long': {'period': 84}, 'RSI': {'period': 10}, 'BollingerBands': {'period': 28, 'std_dev': 2.2892660995016065}, 'MACD': {'fast_period': 10, 'slow_period': 22, 'signal_period': 7}}
  Strategy: {'stop_loss': 0.0030842071457064077, 'take_profit': 0.009831937816302467, 'position_size': 0.6706444754444352}
Erreur dans backtest_strategy: Colonnes manquantes: ['SMA_33', 'SMA_84', 'RSI_10']
[I 2025-11-06 17:41:30,006] Trial 62 finished with value: 0.0 and parameters: {'SMA_short_period': 33, 'SMA_long_period': 84, 'RSI_period': 10, 'BollingerBands_period': 28, 'BollingerBands_std_dev': 2.2892660995016065, 'MACD_fast_period': 10, 'MACD_slow_period': 22, 'MACD_signal_period': 7, 'stop_loss

Best trial: 0. Best value: 0:  88%|████████▊ | 88/100 [00:01<00:00, 66.93it/s, 1.20/3600 seconds]

Erreur dans backtest_strategy: Colonnes manquantes: ['SMA_46', 'SMA_156', 'RSI_8']
[I 2025-11-06 17:41:30,203] Trial 75 finished with value: 0.0 and parameters: {'SMA_short_period': 46, 'SMA_long_period': 156, 'RSI_period': 8, 'BollingerBands_period': 21, 'BollingerBands_std_dev': 2.548097457522383, 'MACD_fast_period': 14, 'MACD_slow_period': 25, 'MACD_signal_period': 7, 'stop_loss': 0.009308814978661683, 'take_profit': 0.01867897549079351, 'position_size': 0.20915614763236356}. Best is trial 0 with value: 0.0.
[<quanta.utils.ta.SMA object at 0x715a9609d110>, <quanta.utils.ta.SMA object at 0x715a95f0cd50>, <quanta.utils.ta.RSI object at 0x715a95f0cf90>, <quanta.utils.ta.BollingerBands object at 0x715a95f0c290>, <quanta.utils.ta.MACD object at 0x715a95f0d1d0>]

Trial 76:
  Indicators: {'SMA_short': {'period': 19}, 'SMA_long': {'period': 131}, 'RSI': {'period': 9}, 'BollingerBands': {'period': 24, 'std_dev': 2.766424358846334}, 'MACD': {'fast_period': 13, 'slow_period': 27, 'signal_perio

Best trial: 0. Best value: 0: 100%|██████████| 100/100 [00:01<00:00, 71.81it/s, 1.39/3600 seconds]


[I 2025-11-06 17:41:30,404] Trial 88 finished with value: 0.0 and parameters: {'SMA_short_period': 24, 'SMA_long_period': 126, 'RSI_period': 7, 'BollingerBands_period': 30, 'BollingerBands_std_dev': 2.162969803181515, 'MACD_fast_period': 9, 'MACD_slow_period': 25, 'MACD_signal_period': 8, 'stop_loss': 0.023279500941018348, 'take_profit': 0.00204481949283492, 'position_size': 0.7788085039481255}. Best is trial 0 with value: 0.0.
[<quanta.utils.ta.SMA object at 0x715a96072f50>, <quanta.utils.ta.SMA object at 0x715a95f2ab50>, <quanta.utils.ta.RSI object at 0x715a95f2ad10>, <quanta.utils.ta.BollingerBands object at 0x715a95f29dd0>, <quanta.utils.ta.MACD object at 0x715a95f2afd0>]

Trial 89:
  Indicators: {'SMA_short': {'period': 46}, 'SMA_long': {'period': 136}, 'RSI': {'period': 14}, 'BollingerBands': {'period': 11, 'std_dev': 2.6590351573023217}, 'MACD': {'fast_period': 14, 'slow_period': 21, 'signal_period': 9}}
  Strategy: {'stop_loss': 0.017091763498461282, 'take_profit': 0.0060570999

In [16]:
best_config

{'indicators': {'SMA_short': {'period': 22},
  'SMA_long': {'period': 192},
  'RSI': {'period': 17},
  'BollingerBands': {'period': 22, 'std_dev': 1.7340279606636548},
  'MACD': {'fast_period': 9, 'slow_period': 20, 'signal_period': 12}},
 'strategy': {'stop_loss': 0.010502105436744277,
  'take_profit': 0.01595857358814127,
  'position_size': 0.1185260448662222}}

In [17]:
# crée un plot avec les meilleurs paramètres
best_indicators = create_indicators_from_params(OPTIMIZATION_CONFIG, {
    'indicators': best_config['indicators'],
    'strategy': best_config['strategy']
})
traces = [
    Candlesticks(),
    Line('hl_avg', name='HL Average', color='purple'),
    Volume()
]
chart_client.plot(df, "AAPL - Best Config", traces=traces, indicators=best_indicators, theme='professional')

Plotting 1491 bars for AAPL - Best Config with x_axis_type='row_nb'
